In [27]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set(font_scale=1.2)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import pylab 
import scipy.stats as stats

from scipy.stats import skew

from copy import deepcopy

import sklearn.model_selection as ms
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

import xgboost 

In [28]:
# Global Setting


# Rand Seed
RandSeed0 = 123;


# Set the threshold for being over-skewed
# (can tune later)
skew0 = 0.6;


#-----------------------------------------------------

# Setting of Flags

# For Section 1: 
# Data Loading ----------------------------------------
#whichDataSet = 0;
whichDataSet = 'w';

# For Section 2: 
# Data Pre-Processing ---------------------------------
use_2_CookedUp_Features = 1;
# Use Bath/Bed and Garage/Bed or not
# 1 means Yes; other numbers mean No. 

use_2_CookedUp_Features_instead = 0;
# Use Bath/Bed and Garage/Bed instead of Bath (Full, Half), Garage
# 1 means Yes; other numbers mean No. 

## 1. Data Loading

In [29]:
if whichDataSet == 0: # Using the original data set 
    train = pd.read_csv('Datasets/train.csv')
    test = pd.read_csv('Datasets/test.csv')
    # Concat. train[no ID column, ... (all columns) ..., no SalePrice column]
    #     with test[no ID column, ... (all columns) ..., no SalePrice column]
    # (Test data has no SalePrice column anyway)
    train_test = pd.concat([train.loc[:,'MSSubClass':'SaleCondition'],
                             test.loc[:,'MSSubClass':'SaleCondition']]);
    print('Train-Test mega dataset shape:',train_test.shape)
##
##
if whichDataSet == 'w': # Using Wenchang's modified data set
    train = pd.read_csv('Datasets/train_Wenchang.csv')
    test = pd.read_csv('Datasets/test_Wenchang.csv')
    train.drop(['Unnamed: 0'], axis=1, inplace=True)
    test.drop(['Unnamed: 0'], axis=1, inplace=True)
    train_test = pd.concat([train.loc[:,'1stFlrSF':'RemodYearDiff'],
                             test.loc[:,'1stFlrSF':'RemodYearDiff']]);
    print('Train-Test mega dataset shape:',train_test.shape)

Train-Test mega dataset shape: (2919, 72)


## 2. Data Pre-Processing
* 2.0 (Optional) Add Cooked_up Features: Bath_Capacity and Parking_Capacity
* 2.1 Some Preliminary Examination and Symmetrization
* 2.2 Fill in NAs
* 2.3 Encode Categorical Features
* 2.4 Set up training and test data matrices

### 2.0 (Optional) Add Cooked_up Features: Bath_Capacitance and Parking_Capacitance

In [30]:
if use_2_CookedUp_Features == 1:
    #
    # Compute Total Bathrooms in a house
    # and set those with zero bathroom with median bathroom number
    train['TotBath'] = train.FullBath + 0.5*train.HalfBath;
    test['TotBath'] = test.FullBath + 0.5*test.HalfBath;
    train_test['TotBath'] = train_test.FullBath + 0.5*train_test.HalfBath;
    #
    train['TotBath'].replace(0, train['TotBath'].median(), inplace=True);
    test['TotBath'].replace(0, test['TotBath'].median(), inplace=True);
    train_test['TotBath'].replace(0, train_test['TotBath'].median(), inplace=True);
    #------------------------------------------------------------------------------
    #
    # Set those with zero bedroom with median bedfroom number 
    # (or set them to one to be conservative)
    train['BedroomAbvGr'].replace(0, train['BedroomAbvGr'].median(), inplace=True);
    test['BedroomAbvGr'].replace(0, test['BedroomAbvGr'].median(), inplace=True);
    train_test['BedroomAbvGr'].replace(0, train_test['BedroomAbvGr'].median(), inplace=True);
    #------------------------------------------------------------------------------
    #
    # Cook-up Feature 1:
    # Bath_Capacitance = TotBath / BedroomAbvGr
    train['Bath_Capacitance'] = train.TotBath / train.BedroomAbvGr
    test['Bath_Capacitance'] = train.TotBath / train.BedroomAbvGr
    train_test['Bath_Capacitance'] = train.TotBath / train.BedroomAbvGr
    #------------------------------------------------------------------------------
    #
    # Cook-up Feature 2: 
    # Parking_Capacitance = TotBath / BedroomAbvGr
    train['Parking_Capacitance'] = train.GarageCars / train.BedroomAbvGr
    test['Parking_Capacitance'] = train.GarageCars / train.BedroomAbvGr
    train_test['Parking_Capacitance'] = train.GarageCars / train.BedroomAbvGr

In [31]:
if use_2_CookedUp_Features_instead == 1:
    #
    # Reasonable dropping ---------------------------------------- 
    #
    train.drop(train[['FullBath', 
                      'HalfBath']], axis=1, inplace=True)
    test.drop(test[['FullBath', 
                    'HalfBath']], axis=1, inplace=True)
    train_test.drop(train_test[['FullBath', 
                                'HalfBath']], axis=1, inplace=True)
    #
    # Dropping that needs more scrutiny ---------------------------------------- 
    #
    #train.drop(train[['TotBath']], axis=1, inplace=True)
    #test.drop(test[['TotBath']], axis=1, inplace=True)
    #train_test.drop(train_test[['TotBath']], axis=1, inplace=True)
    # Dropping that needs more scrutiny ---------------------------------------- 
    #
    #train.drop(train[['GarageCars']], axis=1, inplace=True)
    #test.drop(test[['GarageCars']], axis=1, inplace=True)
    #train_test.drop(train_test[['GarageCars']], axis=1, inplace=True)

### 2.1 (a) Preliminary Examination

In [32]:
# As observed, symmetrize SalePrice via log(1 + ***)
train.SalePrice = np.log(1 + train.SalePrice)

### 2.1 (b) Symmetrization

In [33]:
numeric_features = train_test.dtypes[train_test.dtypes != "object"].index

skewed_features = train[numeric_features].apply(lambda x: skew(x.dropna())) 
skewed_features = skewed_features[abs( skewed_features ) > skew0]
skewed_features = skewed_features.index

In [34]:
train_test[skewed_features] = np.log(1 + train_test[skewed_features])

### 2.2 Encode Categorical Features
#### Using plain and simple one-hot encoding

In [35]:
train_test = pd.get_dummies(train_test)

In [36]:
#train_test.head().T

### 2.3 Fill in NAs

#### Interpolating NAs with the median of each field

In [37]:
if whichDataSet == 0:
    train_test = train_test.fillna(train_test.median())

### 2.4 Set up training and test data matrices

In [38]:
X_train = train_test[:train.shape[0]]
X_test  = train_test[train.shape[0]:]
y_train = train.SalePrice

### 2.5 Create Artificial Train-Test Data Sets via Train-Test-Split
### For model validation purposes

In [39]:
# [X_train_v, X_test_v, y_train_v, y_test_v] 
# all come from the original [X_train, y_train] 
##
## Naming convention: 
## The "_v" in names such as "X_train_v" is 
## to indicate such set is for validation purposes.

X_train_v, X_test_v, y_train_v, y_test_v =\
    ms.train_test_split(deepcopy(X_train),\
                        deepcopy(y_train),\
                        test_size = 1/8,\
                        random_state = RandSeed0)

## 3. Modeling

### 3.1 Linear Regression with Ridge Regularizations where $L(\ \vec{\beta} \ ) = MSE + \alpha \cdot \frac{1}{2} ||\ \vec{\beta}\ ||_{L_{2}}$

In [40]:
# Using the [X_train_v, y_train_v] to search for the optimal hyper-parameter(s)
def rmse_cv(model):
    rmse = np.sqrt( -cross_val_score(model, X_train_v, y_train_v, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

#def rmse(pdSeries0):
#    # Specifically, pdSeries0 is a series of Residuals
#    rmse = np.sqrt( sum(pdSeries0**2) / pdSeries0.size )
#    return(rmse)

# Use the built-in MSE calculator
def rmse(y_predicted, y_actual):
    return( np.sqrt( mean_squared_error(y_actual, y_predicted) ) )


def R2(y_predicted, y_actual):
    # R^2 = 1 - SS_residual / SS_total
    SS_residual = sum((y_predicted - y_actual)**2)
    SS_total = sum((y_actual - y_actual.mean())**2)
    R2 = 1 - SS_residual / SS_total
    return(R2)

In [41]:
# supply a log-ranged alphas from 10^(-2) to 10^(2)
# total: 60 alphas to do CV
alpha_array = np.logspace(-1,2,64)

cv_Ridge = [rmse_cv(Ridge(alpha = Alpha)).mean() for Alpha in alpha_array]

In [42]:
cv_Ridge = pd.Series(cv_Ridge, index = alpha_array)

alpha0 = cv_Ridge[cv_Ridge == cv_Ridge.min()].index[0];
rmse0 = cv_Ridge.min();

In [43]:
# Training the model
model_Ridge = Ridge(alpha0).fit(X_train_v, y_train_v);

In [44]:
# Training Performance
# The results are put in dataframe "predictions_Ridge_train"

predictions_Ridge_train = pd.DataFrame({"Predicted":model_Ridge.predict(X_train_v), 
                                        "Actual":y_train_v});
predictions_Ridge_train["Residual"] = predictions_Ridge_train.Actual - predictions_Ridge_train.Predicted;

R2_Ridge_train = model_Ridge.score(X_train_v, y_train_v);

RMSE_Ridge_train = rmse(predictions_Ridge_train.Actual, predictions_Ridge_train.Predicted);

print('*'*50)
print('Ridge Training Performace: R^2 = {:.4f}'.format(R2_Ridge_train))
print('*'*50)
print('Ridge Training Performace: RMSE = {:.4f}'.format(RMSE_Ridge_train))
print('*'*50)

**************************************************
Ridge Training Performace: R^2 = 0.9233
**************************************************
Ridge Training Performace: RMSE = 0.1107
**************************************************


In [45]:
# Test Performance
# The results are put in dataframe "predictions_Ridge_test"

predictions_Ridge_test = pd.DataFrame({"Predicted":model_Ridge.predict(X_test_v), 
                                       "Actual":y_test_v});
predictions_Ridge_test["Residual"] = predictions_Ridge_test.Actual - predictions_Ridge_test.Predicted;

R2_Ridge_test = model_Ridge.score(X_test_v, y_test_v);

RMSE_Ridge_test = rmse(predictions_Ridge_test.Actual, predictions_Ridge_test.Predicted);

print('*'*50)
print('Ridge Test Performace: R^2 = {:.4f}'.format(R2_Ridge_test))
print('*'*50)
print('Ridge Test Performace: RMSE = {:.4f}'.format(RMSE_Ridge_test))
print('*'*50)

**************************************************
Ridge Test Performace: R^2 = 0.9342
**************************************************
Ridge Test Performace: RMSE = 0.1006
**************************************************


In [55]:
# Generate Test Vector (Ridge)

output_Ridge_test = model_Ridge.predict(X_test);

# de-Logarithm
output_Ridge_test = -1 + np.exp(output_Ridge_test); 

In [57]:
#print(len(output_Ridge_test))

#output_Ridge_test

### 3.2 Try with XGBoost

In [46]:
xgb_paramters = {'learning_rate':[0.005, 0.01, 0.05, 0.1, 0.5],
                 'n_estimators':[50, 100, 200, 500]}

xgb_GridSearch = GridSearchCV(xgboost.XGBRegressor(), param_grid=xgb_paramters)

xgb_GridSearch.fit(X_train_v, y_train_v)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.5], 'n_estimators': [50, 100, 200, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [47]:
learning_rate0 = xgb_GridSearch.best_params_['learning_rate'];
n_estimators0 = xgb_GridSearch.best_params_['n_estimators'];

In [48]:
#xgb_GridSearch.best_score_

0.89167840716541458

In [49]:
'''
xgboost.XGBRegressor(
    max_depth=3, 
    learning_rate=0.1, 
    n_estimators=100, 
    silent=True, 
    objective='reg:linear', 
    booster='gbtree', 
    n_jobs=1, 
    nthread=None, 
    gamma=0, 
    min_child_weight=1, 
    max_delta_step=0, 
    subsample=1, 
    colsample_bytree=1, 
    colsample_bylevel=1, 
    reg_alpha=0, 
    reg_lambda=1, 
    scale_pos_weight=1, 
    base_score=0.5, 
    random_state=0, 
    seed=None, 
    missing=None, 
    **kwargs)
'''


model_xgb = xgboost.XGBRegressor(learning_rate=learning_rate0,
                                 n_estimators=n_estimators0)

model_xgb = model_xgb.fit(X_train_v, y_train_v);

In [50]:
# Training Performance
# The results are put in dataframe "predictions_xgb_train"

predictions_xgb_train = pd.DataFrame({"Predicted":model_xgb.predict(X_train_v), 
                                      "Actual":y_train_v});
predictions_xgb_train["Residual"] = predictions_xgb_train.Actual - predictions_xgb_train.Predicted;

R2_xgb_train = R2(predictions_xgb_train.Predicted, predictions_xgb_train.Actual);

RMSE_xgb_train = rmse(predictions_xgb_train.Actual, predictions_xgb_train.Predicted);

print('*'*50)
print('XGBooster Training Performace: R^2 = {:.4f}'.format(R2_xgb_train))
print('*'*50)
print('XGBooster Training Performace: RMSE = {:.4f}'.format(RMSE_xgb_train))
print('*'*50)

**************************************************
XGBooster Training Performace: R^2 = 0.9756
**************************************************
XGBooster Training Performace: RMSE = 0.0625
**************************************************


In [51]:
# Test Performance
# The results are put in dataframe "predictions_xgb_test"

predictions_xgb_test = pd.DataFrame({"Predicted":model_xgb.predict(X_test_v), 
                                      "Actual":y_test_v});
predictions_xgb_test["Residual"] = predictions_xgb_test.Actual - predictions_xgb_test.Predicted;


R2_xgb_test = R2(predictions_xgb_test.Predicted, predictions_xgb_test.Actual);

RMSE_xgb_test = rmse(predictions_xgb_test.Actual, predictions_xgb_test.Predicted);

print('*'*50)
print('XGBooster Test Performace: R^2 = {:.4f}'.format(R2_xgb_test));
print('*'*50)
print('XGBooster Test Performace: RMSE = {:.4f}'.format(RMSE_xgb_test));
print('*'*50)

**************************************************
XGBooster Test Performace: R^2 = 0.9245
**************************************************
XGBooster Test Performace: RMSE = 0.1077
**************************************************


In [58]:
# Generate Test Vector (XGB)

output_xgb_test = model_xgb.predict(X_test);

# de-Logarithm
output_xgb_test = -1 + np.exp(output_xgb_test); 

In [59]:
#print(len(output_xgb_test))

#output_xgb_test

1459


array([ 124052.9375   ,  159566.109375 ,  183471.59375  , ...,
        157286.53125  ,  118197.6953125,  216752.734375 ], dtype=float32)

### 4. Final Stacking

In [62]:
# use RMSE or use RMSE_to_R2

use_RMSE = 1;

if use_RMSE == 1:
    rho_Ridge = RMSE_Ridge_test;
    rho_xgb = RMSE_xgb_test;
    #rho_RForest = RMSE_RForest_test;
else:
    rho_Ridge = RMSE_Ridge_test / R2_Ridge_test;
    rho_xgb = RMSE_xgb_test / R2_xgb_test;
    #rho_RForest = RMSE_RForest_test / R2_RForest_test;
    
s = rho_Ridge + rho_xgb;

w_Ridge = 1 - rho_Ridge / s;
w_xgb = 1 - rho_xgb / s;

output_final = 1/2 * (w_Ridge * output_Ridge_test + 
                      w_xgb * output_xgb_test)

In [68]:
w_Ridge + w_xgb

1.0